# Task
We want to achieve the following workflow:

Step 1. 
1. Echo a string to msg.txt 
    
2. Echo a number and redirect it to results.txt 
    
Step 2.
1. Duplicate the content in msg.txt two times and redirect it to a new file
    
2. Get the value in results.txt and times the number by 2 and redirect it to results.txt

To construct a workflow in dflow, three parts are needed:
1. Construct OP templates
2. Instantiate the OP template to Step
3. Put steps together and submit the workflow

## Construct OP template

As explained in the [readme](https://github.com/dptech-corp/dflow#122--op-template), OP template is the fundamental component in dflow. For this particular workflow above, we need two OP templates:

For step 1:

In [1]:
from dflow import ShellOPTemplate
step1_templ = ShellOPTemplate(
                name="Hello",
                image="alpine:latest",
                script="echo {{inputs.parameters.msg}} > /tmp/msg.txt && echo {{inputs.parameters.number}} > /tmp/results.txt",
)

This defines the operation to be executed. Next, we need to setup the inputs and outputs for this step.

In [2]:
from dflow import InputParameter, OutputParameter, OutputArtifact
step1_templ.inputs.parameters = {
                            "msg": InputParameter(),
                            "number": InputParameter(),
}
step1_templ.outputs.parameters = {
                            "out_param": OutputParameter(value_from_path="/tmp/results.txt")
}
step1_templ.outputs.artifacts = {
                            "out_art": OutputArtifact(path="/tmp/msg.txt")
}

For step 2: 

In [3]:
step2_templ = ShellOPTemplate(
                name="Duplicate",
                image="alpine:latest",
                script="cat /tmp/foo.txt /tmp/foo.txt > /tmp/bar.txt && echo $(({{inputs.parameters.number}}*2)) > /tmp/results.txt",
)

This defines the operation to be executed. Notice 2 things:
1. We duplicated the content in `/tmp/foo.txt` 2 times, instead of `/tmp/msg.txt` in step 1. This is because OPTemplates are indepednent of each other. To make `/tmp/foo.txt` the same as `/tmp/msg.txt`, we only need to initialize it correctly when instantiating the OP template.
2. We redirected the output of the arithmetic operation to `/tmp/results.txt`. This is not the file appeared in step 1.

In [4]:
from dflow import InputArtifact
step2_templ.inputs.artifacts = {
                            "in_art":InputArtifact(path="/tmp/foo.txt") 
}
step2_templ.inputs.parameters = {
                            "number": InputParameter(),
}
step2_templ.outputs.artifacts = {
                            "out_art": OutputArtifact(path="/tmp/bar.txt")
}
step2_templ.outputs.parameters = {
                            "out_param": OutputParameter(value_from_path="/tmp/results.txt")
}

## Instantiate the OP template to Step

`Step` in the central block for building a workflow. A `Step` is created by instantiating an OP template. When a `Step` is initialized, values of all input parameters and sources of all input artifacts declared in the OP template must be specified.

In [5]:
from dflow import Step

step1 = Step (
            name="step1",
            template=step1_templ,
            parameters={"msg":"HelloWorld!", "number": 1},
)
step2 = Step(
            name="step2",
            template=step2_templ,
            parameters={"number":step1.outputs.parameters["out_param"]},
            artifacts={"in_art":step1.outputs.artifacts["out_art"]},
)

Step 1 takes in two values as parameters: "HelloWorld!" and 1. Step 2 takes the values and files from step 1 as the input parameters and artifacts.

## Put steps together and submit a workflow

We finished building blocks of this workflow. Now we can put them togther

In [6]:
from dflow import Workflow
wf = Workflow(name="helloworld")
wf.add(step1)
wf.add(step2)

This creates a workflow with name "helloworld" and adds two steps in series.

We can then submit the workflow. One caveat: we will get warning about certificiate verification since we haven't yet added cerificate to the address we specified for the UI. To suppress it, we can run the following 

In [7]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
wf.submit()

Another caveat: if you want to rerun the workflow, you need to start a new workflow by reruning `wf = Workflow(name="helloworld")`

# <a name="Argo-UI"></a> Argo UI

After finishing the previous steps, we can access the workflow we just ran on the UI (https://127.0.0.1:2746)

We should see the following once loaded.

<img src="./imgs/argoui_main_page.png" alt="argoUI_mainpage"/>

We can see the workflow we just ran. Left click it then we can see the following.

<img src="./imgs/workflow_overview.png" alt="workflow_overview"/>

This gives us an overview of the workflow. The first node does not do anything. The two following nodes are the steps specified by us. Click on it then we can see more information about each step.

We can access the input/outputs of step 2. We can see the parameters of the step on the UI. We can download `out_art` by clicking the download buttom. 

<img src="./imgs/access_one_node.png" alt="access_one_node"/>

After decompressing it, you should see a file named `bar.txt`. (This is also what we specified). Open it, you should see `HelloWorld!\nHelloWorld!`